# Hands-on example

In [1]:
import os
import warnings
import sys

import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

In [2]:
pd.set_option("display.max_columns", 15)

# 0. The data

In [3]:
data_path = "data/wine-quality.csv"
df = pd.read_csv(data_path)

print(f"df shape -> {df.shape}")
df.head()

df shape -> (4898, 12)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


# 1. Tracking experiments

In [4]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 0.0.0.0 --port 5000
remote_server_uri = "http://0.0.0.0:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env

In [5]:
mlflow.tracking.get_tracking_uri()

'http://0.0.0.0:5000'

In [6]:
exp_name = "ElasticNet_wine1"
mlflow.set_experiment(exp_name)

2022/01/25 15:57:16 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_wine1' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlruns/2', experiment_id='2', lifecycle_stage='active', name='ElasticNet_wine1', tags={}>

In [7]:
!mlflow ui

[2022-01-25 15:57:24 +0530] [167] [INFO] Starting gunicorn 20.1.0
[2022-01-25 15:57:24 +0530] [167] [INFO] Listening at: http://127.0.0.1:5000 (167)
[2022-01-25 15:57:24 +0530] [167] [INFO] Using worker: sync
[2022-01-25 15:57:24 +0530] [168] [INFO] Booting worker with pid: 168
^C
[2022-01-25 15:57:45 +0530] [167] [INFO] Handling signal: int
[2022-01-25 15:57:46 +0530] [168] [INFO] Worker exiting (pid: 168)


In [8]:
def eval_matrics(actual, pred):
    """compute relevant metrics"""
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2= r2_score(actual, pred)
    
    return rmse, mae, r2

def load_data(data_path):
    """load the data and split it into training and test set"""
    df = pd.read_csv(data_path)
    
    #spliting the data into train and test set(0.75 and 0.25 respectively)
    train, test = train_test_split(df, random_state= 42)
    
    train_x = train.drop(["quality"], axis= 1)
    test_x = test.drop(["quality"], axis= 1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    
    return train_x, train_y, test_x, test_y

In [16]:
def train(alpha=0.5, l1_ratio=0.5):
    """train the model with the parameters given"""
    warnings.filterwarnings("ignore")
    np.random.seed(42)
    
    #reed the wwine-quality csv file and make sure that you're running this from the root of MLflow
    data_path = "data/wine-quality.csv"
    
    train_x, train_y, test_x, test_y = load_data(data_path)
    
    #useful for multiple runs
    with mlflow.start_run():
        #execute ElasticNet
        lr = ElasticNet(alpha= alpha, l1_ratio= l1_ratio, random_state= 42)
        lr.fit(train_x, train_y)
        
        #evaluate using mwtrics
        pred_y = lr.predict(test_x)
        (rmse, mae, r2) = eval_matrics(test_y, pred_y)
        
        #print the metrics
        print(f"ElasticNet model (alpha = {alpha}, l1_ratio = {l1_ratio}):")
        print(f"RMSE: {rmse}")
        print(f"MAE: {mae}")
        print(f"R2: {r2}")
        
        #log parameters, metrics and model to MLflow
        mlflow.log_param(key= "alpha", value= alpha)
        mlflow.log_param(key= "l1_ration", value= l1_ratio)
        mlflow.log_metric(key= "rmse", value= rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print(f"Save to {mlflow.get_artifact_uri()}")
        
        mlflow.sklearn.log_model(lr, "model")

In [27]:
train(0.5, 0.5)

ElasticNet model (alpha = 0.5, l1_ratio = 0.5):
RMSE: 0.821195103195106
MAE: 0.6338934920844772
R2: 0.1313572669551294
Save to mlruns/1/6f3a6a194d2a427a87391730fa153988/artifacts


In [28]:
train(0.2, 0.2)

ElasticNet model (alpha = 0.2, l1_ratio = 0.2):
RMSE: 0.7854974519820409
MAE: 0.6171230074647533
R2: 0.20523625056452044
Save to mlruns/1/be4dc08caed046efa2856fa817858ffd/artifacts


In [29]:
train(0.1, 0.1)

ElasticNet model (alpha = 0.1, l1_ratio = 0.1):
RMSE: 0.7801538925222868
MAE: 0.6142807154069012
R2: 0.21601266265188934
Save to mlruns/1/be48e6f00e744d34aded311a8d92783e/artifacts


# 2. Packaging the experiment as a MLflow project as conda env

# 3. Deploy the model

# 4. Tagging runs

In [111]:
from datetime import datetime
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiments = client.list_experiments()

print("\033[4m" + "Experiments:" + "\033[0m" + "\n")
print(*experiments, sep="\n")

print("\n", "-"*100, "\n")

print("\033[4m" + "Run info:" + "\033[0m" + "\n")
print(*client.list_run_infos(1), sep="\n\n")

Experiments:

<Experiment: artifact_location='mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>
<Experiment: artifact_location='mlruns/1', experiment_id='1', lifecycle_stage='active', name='ElasticNet_wine', tags={}>

 ---------------------------------------------------------------------------------------------------- 

Run info:

<RunInfo: artifact_uri='mlruns/1/be48e6f00e744d34aded311a8d92783e/artifacts', end_time=1643012899525, experiment_id='1', lifecycle_stage='active', run_id='be48e6f00e744d34aded311a8d92783e', run_uuid='be48e6f00e744d34aded311a8d92783e', start_time=1643012895834, status='FINISHED', user_id='apurv-dell'>

<RunInfo: artifact_uri='mlruns/1/be4dc08caed046efa2856fa817858ffd/artifacts', end_time=1643012888979, experiment_id='1', lifecycle_stage='active', run_id='be4dc08caed046efa2856fa817858ffd', run_uuid='be4dc08caed046efa2856fa817858ffd', start_time=1643012885548, status='FINISHED', user_id='apurv-dell'>

<RunInfo: artifact_uri='mlruns

In [112]:
# get the run
_run = client.get_run(run_id="be48e6f00e744d34aded311a8d92783e")
print(_run)

<Run: data=<RunData: metrics={'mae': 0.6142807154069012,
 'r2': 0.21601266265188934,
 'rmse': 0.7801538925222868}, params={'alpha': '0.1', 'l1_ration': '0.1'}, tags={'mlflow.log-model.history': '[{"run_id": "be48e6f00e744d34aded311a8d92783e", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2022-01-24 08:28:16.425494", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"loader_module": "mlflow.sklearn", '
                             '"python_version": "3.9.7", "env": "conda.yaml"}, '
                             '"sklearn": {"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.0.1", '
                             '"serialization_format": "cloudpickle"}}}]',
 'mlflow.source.name': '/home/apurv-dell/miniconda3/envs/myenv/lib/python3.9/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'apur

In [122]:
# add a tag to the run
dt = datetime.now().strftime("%d-%m-%Y %H:%M:%S.%f")
client.set_tag(_run.info.run_id, "deployed", dt)

In [17]:
!mlflow ui

[2022-01-27 12:25:21 +0530] [355] [INFO] Starting gunicorn 20.1.0
[2022-01-27 12:25:21 +0530] [355] [INFO] Listening at: http://127.0.0.1:5000 (355)
[2022-01-27 12:25:21 +0530] [355] [INFO] Using worker: sync
[2022-01-27 12:25:21 +0530] [356] [INFO] Booting worker with pid: 356
^C
[2022-01-27 12:25:47 +0530] [355] [INFO] Handling signal: int
[2022-01-27 12:25:47 +0530] [356] [INFO] Worker exiting (pid: 356)
